In [ ]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick
from datetime import datetime
import math
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle

SEED = 1111

rf_model_file = '../../models/best_random_model.pkl'

with open(rf_model_file, 'rb') as file:
    best_random_model = pickle.load(file)
    

# Open the model dataset and the dataset with ALL students
training_dataset = pd.read_csv('../../data/processed/extended_dataset_final.csv', index_col = 0)
training_dummy = pd.get_dummies(training_dataset.drop(['CODIGO', 'PERIODO_ACADEMICO'], axis = 1), drop_first = True)

training_dataset_full = pd.read_csv('../../data/frontend/frontend_dataset.csv', index_col = 0)
training_dummy_full = pd.get_dummies(training_dataset_full.drop(['CODIGO', 'PERIODO_ACADEMICO'], axis=1),drop_first=True)


# Start prediction algorithm
sw = True
while sw == True:

    # Get the student's code
    student_code = input("Please enter a student's code:\n")
    student_code = int(student_code)

    # Get a copy to work on
    student_df = training_dataset_full.copy()

    try:
        # Keep only the selected student
        student_df = student_df[student_df['CODIGO'] == student_code].reset_index()
        student_row = student_df.iloc[student_df['n_semesters'].idxmax():1,:]

        # Transform into dummy array
        student_row_dummy = pd.get_dummies(student_row.drop(['CODIGO', 'PERIODO_ACADEMICO'], axis = 1))
        student_row_dummy = student_row_dummy.reindex(columns = training_dummy.columns, fill_value=0)

        # Prediction
        result = best_random_model.predict_proba(student_row_dummy.drop(['ES_DESERTOR_SI'], axis = 1).values)[0][1]*100
        
        # Print results
        print('')
        print('This student has {:.3f}% probabilities to drop out'.format(result))
        print('')

    except:
        print('Student not found')
        
    # Keep the program running
    sw_2 = True    
    while sw_2 == True:
        next_step = student_code = input("Do you want to check another student? (y/n)\n")

        if next_step == 'y':
            sw_2 = False

        elif next_step == 'n':
            sw_2 = False
            sw = False
            
        else:
            print('')
            print('Unknown command. Try again')

Please enter a student's code:
23
Student not found


In [ ]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick
from datetime import datetime
import math
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import pickle

SEED = 1111

rf_model_file = '../../models/best_random_model.pkl'

with open(rf_model_file, 'rb') as file:
    best_random_model = pickle.load(file)
    

# Open the model dataset and the dataset with ALL students
training_dataset = pd.read_csv('../../data/processed/extended_dataset_final.csv', index_col = 0)
training_dummy = pd.get_dummies(training_dataset.drop(['CODIGO', 'PERIODO_ACADEMICO'], axis = 1), drop_first = True)

training_dataset_full = pd.read_csv('../../data/frontend/frontend_dataset.csv', index_col = 0)
training_dummy_full = pd.get_dummies(training_dataset_full.drop(['CODIGO', 'PERIODO_ACADEMICO'], axis=1),drop_first=True)




# Start prediction algorithm
sw = True
while sw == True:
    
    try:
        # Get the period
        year = input("Please enter a year:\n")
        semester = input('Please specify the semester of the year (1/2) a year:\n')

        if semester == '1':
            period = int(str(year)+'10')
        elif semester == '2':
            period = int(str(year)+'20')

        tresh = float(input("Please select a treshold:\n"))

        # Get a copy to work on
        period_df = training_dataset_full.copy()

        # Keep only the selected student
        period_rows = period_df[period_df['PERIODO_ACADEMICO'] == period].reset_index(drop = True)

        # Transform into dummy array
        period_rows_dummy = pd.get_dummies(period_rows.drop(['CODIGO', 'PERIODO_ACADEMICO'], axis = 1))
        period_rows_dummy = period_rows_dummy.reindex(columns = training_dummy.columns, fill_value=0)

        # Prediction
        result = best_random_model.predict_proba(period_rows_dummy.drop(['ES_DESERTOR_SI'], axis = 1).values)
        dropout_prob = [i[1] for i in result]

        period_rows['dropout_prob'] = dropout_prob

        dropout_period = period_rows[period_rows['dropout_prob']>tresh]

        # Print results
        print('')
        print('There are {} students with more than {}% probabilities to drop out in the selected period'\
                                  .format(len(dropout_period), tresh*100))
        print('')
        
    except:
        print('Period not found')
        
    
    # Keep the program running
    sw_2 = True    
    while sw_2 == True:
        next_step = student_code = input("Do you want to check another student? (y/n)\n")

        if next_step == 'y':
            sw_2 = False

        elif next_step == 'n':
            sw_2 = False
            sw = False
            
        else:
            print('')
            print('Unknown command. Try again')

Please enter a year:
2020
Please specify the semester of the year (1/2) a year:
1
Please select a treshold:
0.72

There are 113 students with more than 72.0% probabilities to drop out in the selected period



In [47]:
display(dropout_period.head(5))

,CODIGO,n_semesters,PERIODO_ACADEMICO,cumulative_credits,real_cumulative_gpa,cumulative_failed,gpa_last_semester,GENERO,EDAD,COLEGIO_PROCEDENCIA,PUNT_TOTAL,PUNT_AREA,ES_DESERTOR,FACULTAD,NOMBRE_PROGRAMA,dropout_prob
4,11312009,8,201510,104.0,3.376923,5.0,3.35,F,28.0,COL. NTRA SRA DEL ROSARIO-CHIQUINQUIRÁ,421,37,SI,FACULTAD DE CIENCIAS ADMINISTRATIVAS Y CONTABLES,ADMINISTRACIÓN DE EMPRESAS,0.792592
11,11312514,6,201510,73.0,3.416438,3.0,3.36,F,25.0,I.E. NACIONALIZADA TECNICA AGROPECUARIA CAMPOH...,43,44,SI,FACULTAD DE CIENCIAS ADMINISTRATIVAS Y CONTABLES,ADMINISTRACIÓN DE EMPRESAS,0.981483
12,11312518,7,201510,97.0,2.969072,11.0,1.65,M,32.0,INSTITUCIÓN EDUCATIVA ANTONIO NARIÑO,414,46,SI,FACULTAD DE CIENCIAS ADMINISTRATIVAS Y CONTABLES,ADMINISTRACIÓN DE EMPRESAS,0.996659
18,11313016,5,201510,94.0,4.151064,0.0,4.13,M,26.0,COLEGIO CRISTIANO FILADELFIA,366,47,SI,FACULTAD DE CIENCIAS ADMINISTRATIVAS Y CONTABLES,ADMINISTRACIÓN DE EMPRESAS,0.773727
19,11313017,6,201510,50.0,2.634000,9.0,0.08,M,32.0,IE ANTONIO RICAURTE,442,48,SI,FACULTAD DE CIENCIAS ADMINISTRATIVAS Y CONTABLES,ADMINISTRACIÓN DE EMPRESAS,0.998391


In [ ]:
common_data = 